In [ ]:
# default_exp mask_transform

# mask_transform

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import os
import numpy as np
import nibabel as nb
from nilearn import image, tests
from fastcore import test
import argparse

## The transformation function

In [ ]:
# exports
def binarize_mask(input_image):
    transform_formula = '((i == 255) * 1).astype(np.uint8)'
    out_image = image.math_img(transform_formula, i=input_image)
    return out_image

In [ ]:
# hide
test_img = np.zeros((10,10))
test_img[3:6, 3:6] = 255.0
test_img = nb.Nifti1Image(test_img, affine = np.eye(4))
out_img = binarize_mask(test_img)

# unit test for dtype in header
test.test_eq(out_img.header['datatype'], 2)
# unit test for transversion
test.test_eq(out_img.get_fdata()[3:6, 3:6], np.ones((3,3)))
# unit test for equal type of array
test.test_eq_type(out_img.get_fdata()[3:6, 3:6], np.ones((3,3), dtype=np.uint8))

## Util to load_data

In [ ]:
#export
def data_loader(inpath):
    return nb.load(inpath)

In [ ]:
#export
def data_saver(nifti, outpath):
    try:
        nb.save(nifti, outpath)
        return True
    except:
        return False

## Path-handling
Could also become a class later.

In [ ]:
#export
def outpath_from_inpath(inpath):
    image_dir = os.path.dirname(inpath)
    return image_dir

In [ ]:
#hide
# unit tests for path handler
pth = 'd:/GithubRepros/nilearn_mask_binarizer/CONTRIBUTING.md'
exppth = 'd:/GithubRepros/nilearn_mask_binarizer'

test.test_eq(outpath_from_inpath(pth), exppth)

In [ ]:
#export
def check_outpath(outpath):
    if os.path.isdir(outpath):
        return True
    else:
        raise Exception('outpath has to be a directory!')

In [ ]:
#hide
pth = 'd:/GithubRepros/nilearn_mask_binarizer\CONTRIBUTING.md'
test.test_eq(check_outpath('..'), True)

with test.exception:
    check_outpath(pth)

In [ ]:
#export
def create_outpath(imagename, outpath, prefix):
    if check_outpath(outpath):
        if prefix is not None:
            return os.path.join(outpath, prefix + imagename)
        else:
            return os.path.join(outpath, imagename)

In [ ]:
#hide
pth = 'd:/GithubRepros/nilearn_mask_binarizer\CONTRIBUTING.md'
outpth = '..'
test.test_eq(create_outpath('CONTRIBUTING.md', outpth, 'ch_'), '..\\ch_CONTRIBUTING.md')

###  Putting path funtions together

In [ ]:
#export
def savingpath(inpath, outpath=None, prefix='bin_'):
    imagename = os.path.basename(inpath)
    
    if outpath is None:
        outpath = outpath_from_inpath(inpath)
    
    save_path = create_outpath(imagename, outpath, prefix)
    
    return save_path

In [ ]:
#hide
pth = 'd:/GithubRepros/nilearn_mask_binarizer\CONTRIBUTING.md'
outpth = '..'
test.test_eq(savingpath(pth, outpth, 'ch_'), '..\\ch_CONTRIBUTING.md')

# Main Function

In [ ]:
#export
def mask_transform(inpath, outpath=None, prefix='bin_'):
    nifti = data_loader(inpath)
    savepath = savingpath(inpath, outpath, prefix)
    nifti_bin = binarize_mask(nifti)
    saving = data_saver(nifti_bin, savepath)
    
    if saving:
        print("Saved image!")
        return 1
    else:
        print("Not saved!")
        return 0

In [ ]:
# export
def nifti_binarizer():
    parser = argparse.ArgumentParser(description='Binarize a nifti-mask from 255 to 1.',
                                     epilog='Operation complete!')

    parser.add_argument(dest='inpath', type=str,
                    help='The input image.')

    parser.add_argument('-o', '--outpath', type=str, 
                         help='The output filename and path', default='')

    parser.add_argument('-p', '--prefix', type=str, 
                         help='The prefix to the filename', 
                         default='bin_')
    
    
    args = parser.parse_args()
    
    if args.outpath == '':
        args.outpath = None
    
    if mask_transform(args.inpath, args.outpath, args.prefix):
        print("Succes!!")
    else:
        print("Fail! Maybe check paths.")
    